In [1]:
import optuna
import torch
import random
import numpy as np

from eeg_snn_encoder.config import PROCESSED_DATA_DIR
from eeg_snn_encoder.dataset import CHBMITDataModule, CHBMITDataset

torch.set_float32_matmul_precision("high")
torch.use_deterministic_algorithms(True, warn_only=True)

torch.manual_seed(47)
random.seed(47)
np.random.seed(47)

2025-06-11 16:21:38.604 | INFO     | eeg_snn_encoder.config:<module>:11 - PROJ_ROOT path is: /workspace/snn-encoder-test


In [2]:
# Load the dataset
dataset = CHBMITDataset(PROCESSED_DATA_DIR / "stft_normalized.h5")

In [3]:
datamodule = CHBMITDataModule(dataset, batch_size=768, worker=0)

In [4]:
tuning_encoder = "dummy"

In [5]:
if tuning_encoder not in ["be", "pe", "poisson", "bsa", "sf", "tbr", "dummy"]:
    raise ValueError(f"Unknown encoder type: {tuning_encoder}")

In [6]:
import os

tuned_study_name = f"model-tuning-{tuning_encoder}-new"

tuned_study = optuna.load_study(
    study_name=tuned_study_name,
        storage=os.environ["OPTUNA_CONN_STRING_CPE"],
)

In [13]:
sorted_trial = sorted(tuned_study.trials, key=lambda t: t.value or float("inf"))

In [20]:
import json

deduped_trials = []

param_set = set()

for trial in sorted_trial:
    param_text = json.dumps(trial.params, sort_keys=True)
    if param_text not in param_set:
        param_set.add(param_text)
        deduped_trials.append(trial)

print(f"Total trials: {len(deduped_trials)}")

Total trials: 304


In [ ]:
from eeg_snn_encoder.tuning import create_objective
from loguru import logger

logger.info(f"Starting tuning for {tuning_encoder}")

sampler = optuna.samplers.TPESampler()
pruner = optuna.pruners.NopPruner()

study = optuna.create_study(
    direction="minimize",
    study_name=f"model-fine-tuning-{tuning_encoder}-new",
    storage=os.environ["OPTUNA_CONN_STRING"],
    load_if_exists=True,
    sampler=sampler,
    pruner=pruner,
)

objective = create_objective(
    encoder_type=tuning_encoder,
    datamodule=datamodule,
    moniter_metric="val_mse",
    moniter_mode="min",
    epochs=100
)

[I 2025-06-11 16:21:54,415] A new study created in RDB with name: model-fine-tuning-dummy-new


2025-06-11 16:21:54.295 | INFO     | __main__:<module>:6 - Starting tuning for dummy


In [ ]:
fine_tune_trials = deduped_trials[:5]

In [ ]:
for trial in fine_tune_trials:
    logger.info(f"Starting trial {trial.number} with value {trial.value}")
    study.enqueue_trial(trial.params)

2025-06-11 16:21:54.540 | INFO     | __main__:<module>:2 - Starting trial 132 with value 0.18061089515686035
2025-06-11 16:21:54.559 | INFO     | __main__:<module>:2 - Starting trial 324 with value 0.18193891644477844
2025-06-11 16:21:54.568 | INFO     | __main__:<module>:2 - Starting trial 178 with value 0.18592298030853271
2025-06-11 16:21:54.575 | INFO     | __main__:<module>:2 - Starting trial 305 with value 0.18592298030853271
2025-06-11 16:21:54.580 | INFO     | __main__:<module>:2 - Starting trial 304 with value 0.19521912932395935
2025-06-11 16:21:54.584 | INFO     | __main__:<module>:2 - Starting trial 314 with value 0.19521912932395935
2025-06-11 16:21:54.588 | INFO     | __main__:<module>:2 - Starting trial 322 with value 0.19521912932395935
2025-06-11 16:21:54.591 | INFO     | __main__:<module>:2 - Starting trial 337 with value 0.19920319318771362
2025-06-11 16:21:54.595 | INFO     | __main__:<module>:2 - Starting trial 91 with value 0.2005312144756317
2025-06-11 16:21:54.5

In [ ]:
try:
    study.optimize(objective, n_trials=len(fine_tune_trials), gc_after_trial=True)
    logger.info(f"Best value for {tuning_encoder}: {study.best_value}")
    logger.info(f"Best params for {tuning_encoder}: {study.best_params}")

    logger.info(f"Complete tuning {tuning_encoder} for {len(fine_tune_trials)} trials")
except Exception as e:
    logger.exception(f"Error occurred during optimization for {tuning_encoder}: {e}")

In [24]:
for i in ["be", "pe", "poisson", "bsa", "sf", "tbr", "dummy"]:
    optuna.copy_study(
        from_study_name=f"model-fine-tuning-{i}-new",
        to_study_name=f"model-fine-tuning-{i}-new",
        from_storage=os.environ["OPTUNA_CONN_STRING"],
        to_storage=os.environ["OPTUNA_CONN_STRING_CPE"],
    )

[I 2025-06-11 21:09:51,004] A new study created in RDB with name: model-fine-tuning-be-new


KeyboardInterrupt: 